# Comparison of observations with predictions from the uncalibrated model

In [ ]:
obs_path = '../data/Matsch B2/observations/obs.csv'
model_path = '../data/Matsch B2/geotop'
depths = [50, 200]

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

from mhpc_project.models import FullModel
from mhpc_project.utils import date_parser, comparison_plot

In [ ]:
observations = pd.read_csv(obs_path,
                           parse_dates=[0],
                           date_parser=date_parser,
                           index_col=0)

In [ ]:
model = FullModel(model_path)
model.settings['SoilPlotDepths'] = depths
model.settings['SoilIceContentProfileFileWriteEnd'] = '"theta_ice"'
model.settings['PointAll'] = True
model.settings['PointOutputFileWriteEnd'] = '"point"'
model.settings['SoilTempProfileFileWriteEnd'] = '"temperature"'

simulation = model()

In [ ]:
for col in observations.columns:
    try:
        desc = col.replace('_', ' ').title()
        comparison_plot(observations[col], simulation[col],desc=desc)
        plt.show()
    except:
        pass